In [88]:
from selenium import webdriver as Webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time


def scrape_yelp_reviews(yelp_url):
    webdriver = _get_webdriver()
    # get business url with halal-relevant reviews only
    webdriver.get(yelp_url + '&q=halal')
    review_num = _get_yelp_reviews_num(webdriver) # a function that will return number of reviews
    print(review_num)
    data = []
    for i in range(1 ,int(review_num / 20) + (review_num % 20 > 0)):
        data.extend(_scrape_yelp_halal_reviews(webdriver))
        webdriver.get(yelp_url + '&start='+str(i*20) + '&q=halal')
    data.extend(_scrape_yelp_halal_reviews(webdriver))
    _close_webdriver(webdriver)
    return data

def _get_webdriver():
    chromedriver_path = '/Users/wesamazaizeh/Desktop/Projects/Chrome_driver/chromedriver_83'
    options = ChromeOptions()
    # will need more configuration when deployed
    options.add_argument('headless')
    options.add_argument('--disable-infobars --disable-extensions')
    webdriver = Webdriver.Chrome(executable_path=chromedriver_path, chrome_options=options)
    return webdriver


def _close_webdriver(webdriver):
    webdriver.close()


def _get_yelp_reviews_num(webdriver):
    review_num_xpath = '//span[contains(text(), "reviews mentioning")]'
    WebDriverWait(webdriver, 10).until(EC.visibility_of_element_located((By.XPATH, review_num_xpath)))
    review_num_str = webdriver.find_element_by_xpath(review_num_xpath).text
    # returns 0 when no reviews are found
    return int(review_num_str.split()[0])


def _scrape_yelp_halal_reviews(webdriver):
    reviews_xpath = '//span[@lang="en"]'
    reviews = webdriver.find_elements_by_xpath(reviews_xpath)
    reviews_list = []
    for review in reviews:
        text = review.text
        reviews_list.append(text)
        # append to database instead
    return reviews_list


In [89]:
google_test_url = 'https://maps.google.com/?cid=4988939335835406517'
yelp_test_url = 'https://www.yelp.com/biz/mia-halal-food-ozone-park?adjust_creative=xl_eXoprFhB3R9K5U7OxqQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xl_eXoprFhB3R9K5U7OxqQ'

data = scrape_yelp_reviews(yelp_test_url)
len(data)

/Users/wesamazaizeh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: use options instead of chrome_options


52


54

In [84]:
len(data)

9328

In [85]:
data

["The food was OK. The employees or the front staff weren't very nice or approachable. Was really hungry and wanted halal food to eat. Definitely feel like I was ripped off by the price. If you need food to take out, there's a truck by the PC Richard's across, go there. Food is about 10x better.",
 "This place has good halal food, but... I came back after about a year and now their prices are expensive!\n\nIt's good, but... It ain't that good.",
 "Went there at 3am last night for a friend's recommendation. He choose this place over Halal Guys and I was amazed of how good of the food they have. Yes, In some way they have better tasting chicken than Halal Guys also their price is so affordable and food portion is very good for that price I paid. They have large menu to choose from and I tried the Chicken over Rice and I have to say that the best Chicken Over rice I have ever tasted. I also have tried their hot wings and they were very big in size. Definitely will go back every time I vis